## HF state

In [ ]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
import time
X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity

symbols = ["H","H","H","H"]
r_bohr = 5.0*1.8897259886
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method = "pyscf")

print('The original hamiltonian is', H)
electrons = 4
orbitals = 8
singles, doubles = qml.qchem.excitations(electrons, orbitals,fermionic=True)
print('Singles are',singles)
print('Doubles are',doubles)
hf_state = qchem.hf_state(electrons, qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(hf_state, electrons, qubits, H):
    # Prepare the Hartree-Fock state
    print('Updated hf_state is', hf_state)
    qml.BasisState(hf_state, wires=range(qubits))

    return qml.expval(H)
print('HF state is', circuit(hf_state, electrons, qubits, H))


## Gradient calculation - Commutator 

In [ ]:
from itertools import chain
X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
electrons = 4
orbitals = 8


singles, doubles = qml.qchem.excitations(electrons, orbitals, fermionic=True)
print(singles)
print(doubles)


#H, qubits = qchem.molecular_hamiltonian(symbols,geometry, active_electrons = 4, active_orbitals = 4,method = "pyscf")
print('The Hamiltonian is ', H)

w2 = []
w1 = []
e= []
o = []

max_value1 = float('-inf')
max_operator1 = None

max_value2 = float('-inf')
max_operator2 = None
print('HF state is', hf_state)
for i in singles:
    print('The current excitation operator is', i)
    w1 = qml.fermi.jordan_wigner(i)
    dev = qml.device("default.qubit", wires=qubits)
    @qml.qnode(dev)
    def commutator(H,w1, hf_state):
        qml.BasisState(hf_state, wires=range(qubits))
        res = qml.commutator(H, w1)
        return qml.expval(res)
    current_value1 = abs(commutator(H, w1, hf_state))
    print(f'The expectation value of {i} is', current_value1)

    if current_value1 > max_value1:
        max_value1 = current_value1
        max_operator1 = i
print(f"The highest single operator value is {max_value1} for operator {max_operator1}")
for j in doubles:
    print('The current double excitation operator is', j)
    w2 = qml.fermi.jordan_wigner(j)
    dev = qml.device("default.qubit", wires=qubits)
    @qml.qnode(dev)
    def commutator(H,w2, hf_state):
        qml.BasisState(hf_state, wires=range(qubits))
        res = qml.commutator(H, w2)
        return qml.expval(res)
    current_value2 = abs(commutator(H, w2,hf_state))
    print(f'The expectation value of {j} is', commutator(H, w2,hf_state))

    if current_value2 > max_value2:
        max_value2 = current_value2
        max_operator2 = j
print(f"The highest double excitation operator value is {max_value2} for operator {max_operator2}")   

if max_value2 > max_value1:
    e = max_value2
    o = max_operator2
    o_str = str(o) 
    print(f"The highest double excitation operator value is {e} for operator {o_str}")  
else:
    e = max_value1
    o = max_operator1
    o_str = str(o) 
    print(f"The highest single operator value is {e} for operator {o_str}")



## Splitting the excitation to wires 

In [ ]:
print('The operator is', o_str)
print(o_str)
ash_excitation = []

import re


# Regular expression to extract the numbers inside parentheses
indices = re.findall(r'\((\d+)\)', o_str)

# Convert the extracted strings to integers
excitations= [int(index) for index in indices]

# Print the result
print(excitations)

print('The length of the excitation is',len(excitations))

ash_excitation.append(excitations)
print('The length of ash excitation is',len(ash_excitation))
print('The no of qubits are', qubits)

#The order they have is p,q,r,s = [2,3,6,7]
# wires 1 = [s,r] = [7,6] = occupied orbitals
# wires 2 = [q,p] = [3,2] = unoccupied orbitals

#Updated order they have is p,q,r,s = [7,6,3,2]
# wires 1 = [s,r] = [2,3] = occupied orbitals
# wires 2 = [q,p] = [6,7] = unoccupied orbitals

wires1 = []
wires2 = []
wires = []
if len(excitations) == 4:
    wires2 = excitations[:2][::-1]  # Reverse the first two elements [7, 6] → [6, 7]
    wires1 = excitations[2:][::-1]  
else:
    wires = excitations     


print("wires2:", wires2)
print("wires1:", wires1)
print("wires are", wires)

#single excitation
# indices 𝑟 and 𝑝 run over the occupied and unoccupied molecular orbitals, 
# r = occupied p = unoccupied
# wires template = [r,p] = [occupied, unoccupied] = [4,0]
# when changed to original : [occ, unocc] = [0,4]





In [ ]:
print(ash_excitation)
print(wires1)
print(wires2)

In [ ]:
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)
params = np.zeros(len(ash_excitation), requires_grad = True)
print('The parameters that are going in ', params)
fparams = []


    
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(params, excitations, hf_state, qubits, H, wires1, wires2, wires):
    qml.BasisState(hf_state, wires=range(qubits))
    for i, excitation in enumerate(excitations):
        if len(excitations) == 4:
            #wires1 =  wires1#Occupied orbitals
            #wires2 = wires2 #Unoccupied orbitals
            qml.FermionicDoubleExcitation(weight = params[i], wires1=wires1, wires2=wires2)
            return qml.expval(H)
        else:
            qml.BasisState(hf_state, wires=range(qubits))
            qml.FermionicSingleExcitation(weight= params[i], wires = excitations)
            return qml.expval(H)

cost_fn = qml.QNode(circuit, dev,interface="autograd", diff_method="adjoint")
for n in range(100):
    print(f'Each step,the iteration is {n} and the parameter is {params}')
    params, energy = optimizer.step_and_cost(cost_fn, params,excitations=excitations, hf_state=hf_state, qubits=qubits, 
                                             H=H, wires1=wires1,wires2=wires2, wires=wires )
    if n % 1 == 0:
        print("step = {:},  E = {:.8f} Ha".format(n, energy))
fparams.append(params)

    

## So out of the first excitation 

In [ ]:
print('The parameters are', params)
print('Excitation which went in is', ash_excitation)
print('Wires 1, the occupied orbitals are', wires1)
print('Wires 2, the unoccupied orbitals are', wires2)
print(excitations)

## Making the next state

In [ ]:
import pennylane as qml

#w1 = qml.fermi.jordan_wigner(i)
print('Hartree fock state',hf_state)
print('Excitation that is going to be added', excitations)
print('Parameters that are going in', params)

dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def summa(hf_state, excitations, wires1, wires2, params):
        for i, excitation in enumerate(excitations):
                if len(excitations) == 4:
                        print('The double excitation going in ', excitations)
                        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #HF state
                        print('The params that is going in ', params[i])
                        qml.FermionicDoubleExcitation(weight = params[i], wires1=wires1, wires2=wires2)
                        return qml.state()
                #else:
                 #Add single excitation        
ostate = summa(hf_state, excitations, wires1, wires2, params)

dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def summa1(ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        return qml.expval(H)


print('After applying double excitation operator, the circuit is:\n')
print(qml.draw(summa, level="device", max_length=100)(hf_state, excitations, wires1, wires2, params))

print('Energy after applying the excitation is', summa1(ostate))


## After adding the excitation, the state is called ostate

## Going to do gradient calculation again

In [ ]:
from itertools import chain
X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
electrons = 4
orbitals = 8


singles, doubles = qml.qchem.excitations(electrons, orbitals, fermionic=True)
print(singles)
print(doubles)




w2 = []
w1 = []
e= []
o = []

max_value1 = float('-inf')
max_operator1 = None

max_value2 = float('-inf')
max_operator2 = None

print('After adding excitation state  is', ostate)
for i in singles:
    print('The current excitation operator is', i)
    w1 = qml.fermi.jordan_wigner(i)
    dev = qml.device("default.qubit", wires=qubits)
    @qml.qnode(dev)
    def commutator(H,w1, ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        res = qml.commutator(H, w1)
        return qml.expval(res)
    current_value1 = abs(commutator(H, w1, ostate))
    print(f'The expectation value of {i} is', current_value1)

    if current_value1 > max_value1:
        max_value1 = current_value1
        max_operator1 = i
print(f"The highest single operator value is {max_value1} for operator {max_operator1}")
for j in doubles:
    print('The current double excitation operator is', j)
    w2 = qml.fermi.jordan_wigner(j)
    dev = qml.device("default.qubit", wires=qubits)
    @qml.qnode(dev)
    def commutator(H,w2, ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        res = qml.commutator(H, w2)
        return qml.expval(res)
    current_value2 = abs(commutator(H, w2,ostate))
    print(f'The expectation value of {j} is', commutator(H, w2,ostate))

    if current_value2 > max_value2:
        max_value2 = current_value2
        max_operator2 = j
print(f"The highest double excitation operator value is {max_value2} for operator {max_operator2}")   

if max_value2 > max_value1:
    e1 = max_value2
    o1 = max_operator2
    o_str1 = str(o1) 
    print(f"The highest double excitation operator value is {e1} for operator {o_str1}")  
else:
    e1 = max_value1
    o1 = max_operator1
    o_str1 = str(o1) 
    print(f"The highest single operator value is {e1} for operator {o_str1}")

#dummy = qml.qchem.excitations_to_wires(e)
#print(qml.draw_mpl(commutator)(H,w2,hf_state)) 

# Original a⁺(0) a⁺(1) a(4) a(5)
#0.15496366291127073 for operator a⁺(7) a⁺(6) a(3) a(2) - Mine
#Original 0.15496366291127073 for operator a⁺(2) a⁺(3) a(6) a(7)

In [ ]:
print('The operator is', o_str1)
print(o_str1)


import re

# Regular expression to extract the numbers inside parentheses
indices = re.findall(r'\((\d+)\)', o_str1)

# Convert the extracted strings to integers
excitations1= [int(index) for index in indices]

# Print the result
print(excitations1)

print('The length of the excitation is',len(excitations1))

ash_excitation.append(excitations1)
print('The length of ash excitation is',len(ash_excitation))
print('Excitations in ash_excitation are', ash_excitation)


#The order they have is p,q,r,s = [2,3,6,7]
# wires 1 = [s,r] = [7,6] = occupied orbitals
# wires 2 = [q,p] = [3,2] = unoccupied orbitals

#Updated order they have is p,q,r,s = [7,6,3,2]
# wires 1 = [s,r] = [2,3] = occupied orbitals
# wires 2 = [q,p] = [6,7] = unoccupied orbitals

wires3 = []
wires4 = []
wires = []
if len(excitations1) == 4:
    wires4 = excitations1[:2][::-1]  # Reverse the first two elements [7, 6] → [6, 7]
    wires3 = excitations1[2:][::-1]  
else:
    wires = excitations1   


print("wires4:", wires4)
print("wires3:", wires3)
print("wires are", wires)

#single excitation
# indices 𝑟 and 𝑝 run over the occupied and unoccupied molecular orbitals, 
# r = occupied p = unoccupied
# wires template = [r,p] = [occupied, unoccupied] = [4,0]
# when changed to original : [occ, unocc] = [0,4]





## Energy calculation with ostate

In [ ]:
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)
params = np.zeros(len(ash_excitation), requires_grad = True)
print('The parameters that are going in ', params)

params1 = 0.0
params2 = 0.0


#@qml.qnode(dev)
#def circuit(params, excitations1, hf_state, qubits, H, wires1, wires2, wires, wires3, wires4):
#    qml.BasisState(hf_state, wires=range(qubits))  # Prepare initial Hartree-Fock state
#    for i, excitation in enumerate(ash_excitation):
#        if len(excitation) == 4:
#            qml.FermionicDoubleExcitation(weight=params1, wires1=wires1, wires2=wires2)
#            qml.FermionicDoubleExcitation(weight=params2, wires1=wires3, wires2=wires4)
#        else:
#            qml.FermionicSingleExcitation(weight=params[i], wires=excitations1)
#    return qml.expval(H)




    
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(params, excitations1, hf_state, qubits, H, wires1, wires2, wires, wires3, wires4):
    qml.BasisState(hf_state, wires=range(qubits))
    for i, excitations in enumerate(ash_excitation):
        if len(excitations) == 4:
            #wires1 =  wires1#Occupied orbitals
            #wires2 = wires2 #Unoccupied orbitals
            print('The present state of the excitation is',excitations)
            print('The parameters are', params[i])
            
            qml.FermionicDoubleExcitation(weight = params[i], wires1=wires1, wires2=wires2)
            print('After adding the first excitation, the params are', params[i+1])
            qml.FermionicDoubleExcitation(weight = params[i+1], wires1=wires3, wires2=wires4)
            print('After adding the second excitation, the wires are', wires1)
            return qml.expval(H)
        else:
            #Need to add single excitation properly
            #qml.StatePrep(hf_state, wires=range(qubits))
            qml.FermionicSingleExcitation(weight= params[i], wires = excitations1)
            return qml.expval(H)

cost_fn = qml.QNode(circuit, dev,interface="autograd", diff_method="adjoint")
for n in range(100):
    print(f'Each step,the iteration is {n} and the parameter is {params} and wires are {wires}')
    params, energy = optimizer.step_and_cost(cost_fn, params,excitations1=excitations1, hf_state=hf_state, qubits=qubits, 
                                             H=H, wires1=wires1,wires2=wires2, wires=wires, wires3=wires3, wires4=wires4 )
    if n % 2 == 0:
        print("step = {:},  E = {:.8f} Ha".format(n, energy))


#result = circuit(params, excitations1, hf_state, qubits, H, wires1, wires2, wires1, wires3, wires4)   

# Number of optimization steps 
#steps = 100 # Optimize the parameters 
#for step in range(steps): 
#    params, cost = optimizer.step_and_cost(cost_fn, params, excitations1, hf_state, qubits, H, wires1, wires2, wires, wires3, wires4) 
#    print(f"Step {step+1}, Cost: {cost}") # The optimized parameters print("Optimized parameters:"

In [ ]:
A5 =  -1.53547475
A3 = -1.56946035
A1 = -2.14728938

In [ ]:



ash_excitation = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [7, 3]]

print(ash_excitation)

optimizer = qml.GradientDescentOptimizer(stepsize=0.5)

params = np.zeros(len(ash_excitation), requires_grad = True)
#params = np.tensor((-0.59709303, 0.96593200, -0.74237984, 0.86758707, 0.01052714, 0.01048673), requires_grad = False)
print('The parameters that are going in ', params)









dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(params, ash_excitation, hf_state, qubits, H):
    print('HF state', hf_state)
    qml.BasisState(hf_state, wires=range(qubits))
    for i, excitations in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            print('The present state of the ash excitation is', ash_excitation[i])
            print('The parameters are', params[i])
            qml.DoubleExcitation(params[i], wires=ash_excitation[i])
            #qml.FermionicDoubleExcitation(weight = params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])

        else:
            #Need to add single excitation properly
            print('Single excitations coming in is', ash_excitation[i])
            print('Single excitations params:', params[i])
            #qml.FermionicSingleExcitation(weight= params[i], wires = [0,3,5,6])
            qml.SingleExcitation(params[i], wires=ash_excitation[i])
    return qml.expval(H)


cost_fn = qml.QNode(circuit, dev,interface="autograd", diff_method="adjoint")
for n in range(200):
    #print(f'Each step,the iteration is {n} and the parameter is {params}')
    params, energy = optimizer.step_and_cost(cost_fn, params, ash_excitation=ash_excitation, hf_state=hf_state, qubits=qubits,H=H)
    if n % 5 == 0:
        print("step = {:},  E = {:.8f} Ha".format(n, energy))




## Using L-BFGS - scipy minimize

In [ ]:
import numpy as np
import pennylane as qml
from scipy.optimize import minimize

# Define the Ash excitation structure (as per your example)
ash_excitation = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [7, 3], [0,4]]


dev = qml.device("default.qubit", wires=qubits)

# QNode to evaluate energy
@qml.qnode(dev)
def circuit(params, ash_excitation, hf_state, qubits, H):
    qml.BasisState(hf_state, wires=range(qubits))
    for i, excitations in enumerate(ash_excitation):
        if len(excitations) == 4:
            qml.DoubleExcitation(params[i], wires=excitations)
        else:
            print(f'Single excitations coming in is {excitations} and parameters are {params[i]}')
            qml.SingleExcitation(params[i], wires=excitations)
    return qml.expval(H)


#params = np.random.rand(len(ash_excitation))
# Define the cost function that returns the energy and its gradient
def cost_fn(params, ash_excitation, hf_state, qubits, H):
    # Compute the energy
    energy = circuit(params, ash_excitation, hf_state, qubits, H)
    
    # Compute the gradient of the energy with respect to the parameters
    #grad = qml.grad(circuit)(params, ash_excitation, hf_state, qubits, H)
    
    # Debugging: Print the gradient to ensure it is computed correctly
    #print("Energy:", energy)
    #print("Gradient:", grad)
    #print("Gradient shape:", grad.shape if hasattr(grad, 'shape') else "No shape attribute")
    return energy

# Initial parameters (use zeros or random values)
params_initial = np.zeros(len(ash_excitation))


# Function to minimize
def minimize_function(params):
    energy= cost_fn(params, ash_excitation, hf_state, qubits, H)
    return energy

# Using scipy.optimize.minimize with L-BFGS-B
result = minimize(
    fun=minimize_function,   # The cost function
    x0=params_initial,       # Initial guess
    #jac=False,                 # Enable gradient computation
    method='L-BFGS-B',       # Use L-BFGS-B optimizer
    options={'disp': 1, 'maxiter': 500}  # Display options and max iterations
)

# Extract the result
optimized_params = result.x
minimized_energy = result.fun

print(f"Optimized parameters: {optimized_params}")
print(f"Minimized energy: {minimized_energy}")


In [ ]:
print(params)
#-8.44368162e-01 -1.36552979e+00 -1.04955949e+00 -1.22675429e+00 -3.17765691e-05
#-8.44401862e-01 -1.36603754e+00 -1.04986704e+00 -1.22695951e+00 -2.67572595e-05

In [ ]:
Nick = -1.855435678084
GD = -1.85541375
Nick1 = -1.85546081

GD - Nick1

## Trying to use Jax

In [ ]:
import jax
from jax import numpy as jnp
import pennylane as qml
import optax
import jax.numpy as jnp
from pennylane import qchem

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity

symbols = ["H","H","H","H"]
r_bohr = 5.0*1.8897259886
coordinates = jnp.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method = "pyscf")

print('The original hamiltonian is', H)
electrons = 4
orbitals = 8
singles, doubles = qml.qchem.excitations(electrons, orbitals,fermionic=True)
print('Singles are',singles)
print('Doubles are',doubles)
hf_state = qchem.hf_state(electrons, qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")

dev = qml.device("default.qubit", wires=qubits)
#@jax.jit
@qml.qnode(dev, interface="jax")
def circuit(hf_state, electrons, qubits, H):
    # Prepare the Hartree-Fock state
    print('Updated hf_state is', hf_state)
    #qml.BasisState(hf_state, wires=range(qubits))
    [qml.PauliX(i) for i in jnp.nonzero(hf_state)[0]] #HF state

    return qml.expval(H)
print('HF state is', circuit(hf_state, electrons, qubits, H))

#dev = qml.device("default.qubit", wires=2)

#@jax.jit
#@qml.qnode(dev, interface="jax")
def circuit(param):
    qml.RX(param, wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(0))

optimizer = optax.adam(learning_rate=0.1)
params = jnp.array(0.123)
opt_state = optimizer.init(params)

for i in range(20):
    cost, grad_circuit = jax.value_and_grad(circuit)(params)
    updates, opt_state = optimizer.update(grad_circuit, opt_state)
    params = optax.apply_updates(params, updates)
    print(f"step {i}, cost {cost}")

## Trying to generate the Hamiltonian using 1 body and 2 body electronic integrals

In [ ]:
from pyscf import gto, ao2mo, scf
import numpy as np

r_bohr = 5.0*1.8897259886
mol_pyscf = gto.M(atom = '''H 0.00 0.0 9.4486299;;
                            H  0.0  0.0  18.897259886;
                            H  0.0  0.0  28.3458898;
                            H  0.0 0.0  37.794519772''', basis = 'sto-6g', unit = 'bohr')
rhf = scf.RHF(mol_pyscf)
energy = rhf.kernel()

one_ao = mol_pyscf.intor_symmetric('int1e_kin') + mol_pyscf.intor_symmetric('int1e_nuc')
two_ao = mol_pyscf.intor('int2e_sph')

#print('1 body',one_ao)
#print('2 body',two_ao)

one_mo = np.einsum('pi,pq,qj->ij', rhf.mo_coeff, one_ao, rhf.mo_coeff)
two_mo = ao2mo.incore.full(two_ao, rhf.mo_coeff)

two_mo = np.swapaxes(two_mo, 1, 3)

core_constant = np.array([rhf.energy_nuc()])
#print(core_constant)

H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, two_mo)
#print(H_fermionic)
H = qml.jordan_wigner(H_fermionic)
print('Hamiltonian is ',H)


## Introduction of symmetry and building the Hamiltonian

In [1]:
from pyscf import gto, ao2mo, scf
import numpy as np
import pennylane as qml

mol = gto.Mole()
mol.build(atom = f'''H 0 0 9.4486299; H  0 0 18.897259886; H 0 0 28.3458898; H 0 0 37.794519772''', symmetry='d2h', basis='sto-6g', unit='Bohr')

rhf = scf.RHF(mol)
energy = rhf.kernel()

one_ao = mol.intor_symmetric('int1e_kin') + mol.intor_symmetric('int1e_nuc')
two_ao = mol.intor('int2e_sph')

#print('1 body',one_ao)
#print('2 body',two_ao)

one_mo = np.einsum('pi,pq,qj->ij', rhf.mo_coeff, one_ao, rhf.mo_coeff)
two_mo = ao2mo.incore.full(two_ao, rhf.mo_coeff)

two_mo = np.swapaxes(two_mo, 1, 3)

core_constant = np.array([rhf.energy_nuc()])
#print(core_constant)

H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, two_mo)
#print(H_fermionic)
Hs = qml.jordan_wigner(H_fermionic)
print('Hamiltonian is ',Hs)

converged SCF energy = -1.21602385868408
Hamiltonian is  (-1.1091571486954652+0j) * I(0) + (0.041963259056641014+0j) * Z(0) + (6.339400231511732e-05+0j) * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + (6.339400231511732e-05+0j) * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + (0.04183466727290516+0j) * Z(2) + (0.015433496931262794+0j) * (Z(0) @ Z(2)) + (6.344559491165279e-05+0j) * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + (6.344559491165279e-05+0j) * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + (-0.004378038552059337+0j) * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + (-0.004378038552059337+0j) * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + (0.041705055937434295+0j) * Z(4) + (0.019843364944921563+0j) * (Z(0) @ Z(4)) + (0.04157626163871023+0j) * Z(6) + (0.022112211900387833+0j) * (Z(0) @ Z(6)) + (0.04196325905664093+0j) * Z(1) + (0.06277200456493115+0j) * (Z(0) @ Z(1)) + (0.0025025661539114395+0j) * (Y(0) @ Z(2) @ Z(3) @ Y(4)) + (0.0025025661539114395+0j) * (X(0) @ Z(2) @ Z(3) @ X(4)) + (0.0473129501633146+0j) * (Y(0) @ 

In [5]:
import numpy as np
import pennylane as qml
from scipy.optimize import minimize
from pennylane import qchem

# Define the Ash excitation structure (as per your example)
ash_excitation = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [3, 7], [0,4]]
qubits = 8
electrons = 4
hf_state = qchem.hf_state(electrons, qubits)


dev = qml.device("default.qubit", wires=qubits)

# QNode to evaluate energy
@qml.qnode(dev)
def circuit(params, ash_excitation, hf_state, qubits, Hs):
    qml.BasisState(hf_state, wires=range(qubits))
    for i, excitations in enumerate(ash_excitation):
        if len(excitations) == 4:
            qml.DoubleExcitation(params[i], wires=excitations)
        elif len(excitations) == 2:
            print(f'Single excitations coming in is {excitations} and parameters are {params[i]}')
            #qml.SingleExcitation(params[i], wires=excitations)
            qml.FermionicSingleExcitation(weight= params[i], wires = excitations)
    return qml.expval(Hs)



# Initial parameters (use zeros or random values)
params_initial = np.zeros(len(ash_excitation))
print(params_initial)


# Function to minimize
def minimize_function(params):
    energy= circuit(params, ash_excitation, hf_state, qubits, Hs)
    return energy

# Using scipy.optimize.minimize with L-BFGS-B
#result = minimize(
#    fun=minimize_function,   # The cost function
#    x0=params_initial,       # Initial guess
#    #jac=False,                 # Enable gradient computation
##    method='L-BFGS-B',       # Use L-BFGS-B optimizer
    
#    options={'disp': 1, 'maxiter': 500, 'gtol' : 1e-12, 'iprint': 101, 'maxcor':100}  # Display options and max iterations
#)

result = minimize(
    fun=minimize_function,   # The cost function
    x0=params_initial,       # Initial guess
    #jac=False,                 # Enable gradient computation
    method='Powell',       # Use L-BFGS-B optimizer
    #disp = True,
    options={'disp': 'True' , 'maxiter': 1000, 'rhobeg': 9.0, 'tol': 1e-12}  # Display options and max iterations
)

# Extract the result
optimized_params = result.x
minimized_energy = result.fun

print(f"Optimized parameters: {optimized_params}")
print(f"Minimized energy: {minimized_energy}")
#5.678704E-03 -5.679299E-03

[0. 0. 0. 0. 0. 0.]
Single excitations coming in is [3, 7] and parameters are 0.0
Single excitations coming in is [0, 4] and parameters are 0.0
Single excitations coming in is [3, 7] and parameters are 0.0
Single excitations coming in is [0, 4] and parameters are 0.0
Single excitations coming in is [3, 7] and parameters are 0.0
Single excitations coming in is [0, 4] and parameters are 0.0


/var/folders/tg/76l0rq6n6m3bzwcmn22dk8_59wwtc3/T/ipykernel_6984/1087325052.py:50: OptimizeWarning: Unknown solver options: rhobeg, tol
  result = minimize(


Single excitations coming in is [3, 7] and parameters are 0.0
Single excitations coming in is [0, 4] and parameters are 0.0
Single excitations coming in is [3, 7] and parameters are 0.0
Single excitations coming in is [0, 4] and parameters are 0.0
Single excitations coming in is [3, 7] and parameters are 0.0
Single excitations coming in is [0, 4] and parameters are 0.0
Single excitations coming in is [3, 7] and parameters are 0.0
Single excitations coming in is [0, 4] and parameters are 0.0
Single excitations coming in is [3, 7] and parameters are 0.0
Single excitations coming in is [0, 4] and parameters are 0.0
Single excitations coming in is [3, 7] and parameters are 0.0
Single excitations coming in is [0, 4] and parameters are 0.0
Single excitations coming in is [3, 7] and parameters are 0.0
Single excitations coming in is [0, 4] and parameters are 0.0
Single excitations coming in is [3, 7] and parameters are 0.0
Single excitations coming in is [0, 4] and parameters are 0.0
Single e

In [ ]:
-1.85541375